# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Business Problem section 

#### Background

According to Bloomberg News, the London Housing Market is now facing a number of different problems, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of Brexit. More specifically, four problem areas suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales in addition to the brexit situation around.

#### Business Problem

In this scenario,adoption of a  machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions could be of help. 

Business problem: How could we provide support to homebuyers in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

Proposed Solution:
we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, grocery stores,hospitals and recreation facilities.

##  Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). 


## Methodology section

The Methodology section will describe the main components of our analysis and predication system.
The Methodology section comprises four stages: 
    1. Collect Inspection Data
    2. Explore and Understand Data
    3. Data preparation and preprocessing 
    4. Modeling

#### 1. Collect Inspection Data

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - anaconda/linux-64::grpcio==1.16.1=py36hf8bcb03_1 -> openssl[version='>=1.1.1,<1.1.2.0a0']

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


WARNING conda.base.context:use_only_tar_bz2(632): Co

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

#### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows: 

In [3]:
df_ppd.head(5) 

,{79A74E21-C934-1289-E053-6B04A8C01627},177000,2018-09-21 00:00,LE4 6EE,S,N,F,201,Unnamed: 8,BELPER STREET,Unnamed: 10,LEICESTER,LEICESTER.1,LEICESTER.2,A,A.1
0,{79A74E21-C935-1289-E053-6B04A8C01627},90000,2018-10-01 00:00,LE18 2AE,F,N,L,27,NaN,ELIZABETH COURT,NaN,WIGSTON,OADBY AND WIGSTON,LEICESTERSHIRE,A,A
1,{79A74E21-C936-1289-E053-6B04A8C01627},375000,2018-10-04 00:00,LE11 3HG,D,N,F,6,NaN,GOLDFINCH CLOSE,NaN,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A
2,{79A74E21-C937-1289-E053-6B04A8C01627},142500,2018-10-08 00:00,LE3 6UY,S,N,F,19,NaN,PINEHURST CLOSE,NaN,LEICESTER,LEICESTER,LEICESTER,A,A
3,{79A74E21-C938-1289-E053-6B04A8C01627},157500,2018-10-22 00:00,LE13 0JH,S,N,F,103,NaN,WEST AVENUE,NaN,MELTON MOWBRAY,MELTON,LEICESTERSHIRE,A,A
4,{79A74E21-C939-1289-E053-6B04A8C01627},192500,2018-10-02 00:00,LE12 7UT,T,N,F,26,NaN,MELODY DRIVE,SILEBY,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A


In [4]:
df_ppd.shape

(1017356, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly. 

#### 3. Data preparation and preprocessing

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps: 
- Rename the column names 
- Format the date column 
- Sort data by date of sale 
- Select data only for the city of London 
- Make a list of street names in London 
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names 
- Join the data to find the coordinates of locations which fit into client's budget
- Plot recommended locations on London map along with current market prices






In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

# Display the dataframe
df_affordable

,Street,Avg_Price
195,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
420,APPLEGARTH ROAD,2.400000e+06
698,AYLESTONE AVENUE,2.286667e+06
851,BARONSMEAD ROAD,2.375000e+06
975,BEAUCLERC ROAD,2.480000e+06
1096,BELVEDERE DRIVE,2.340000e+06
1209,BICKENHALL STREET,2.208500e+06
1247,BIRCHLANDS AVENUE,2.217000e+06


In [7]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [8]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 195
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 195, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 420
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 420, dtype: object
item.Street only: APPLEGARTH ROAD
index: 698
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 698, dtype: object
item.Street only: AYLESTONE AVENUE
index: 851
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 851, dtype: object
item.Street only: BARONSMEAD ROAD
index: 975
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 975, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1096
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name

In [ ]:
geolocator = Nominatim()


In [ ]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [11]:
df_affordable

,Street,Avg_Price,city_coord
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)"
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1096,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)"
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"


In [ ]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

In [13]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)",53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)",38.201316,-84.623076
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468


In [14]:
df = df_affordable.drop(columns=['city_coord'])


In [15]:
df

,Street,Avg_Price,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076
1209,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
